# データ準備

## Excelシートからcsv形式で保存

In [30]:
import pandas as pd

path="C:\Code\Python_code\_git\\voice\音声データ.xlsx"

input_file=pd.ExcelFile(path)
sheet=input_file.sheet_names

for i in range(0,len(sheet)):

        data=pd.read_excel(path,i,index_col=None)
        data.to_csv(sheet[i]+".csv",encoding="shift_jis")

## csvからデータを取り出す。

In [31]:
male_train=pd.read_csv("C:\Code\Python_code\_git\\voice\A-男性.csv")
female_train=pd.read_csv("C:\Code\Python_code\_git\\voice\A-女性.csv")

In [32]:
male_train=male_train[["f0","f1","f2","f3","f4"]]
female_train=female_train[["f0","f1","f2","f3","f4"]]
male_train["Sex"]=True
female_train["Sex"]=False

In [33]:
train_data=pd.concat([male_train,female_train],axis=0)
train_data.reset_index(drop=True,inplace=True)

## データの分割

In [34]:
from sklearn.model_selection import train_test_split

X=train_data.drop("Sex",axis=1)
y=train_data["Sex"]

X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

## 標準化

In [35]:
from sklearn.preprocessing import StandardScaler

scaler=StandardScaler()

X_train=scaler.fit_transform(X_train)
X_test=scaler.fit_transform(X_test)

# ランダムフォレスト

## モデル作成と予測

In [36]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

model=RandomForestClassifier(n_estimators=100,random_state=42)
model.fit(X_train,y_train)

y_pred=model.predict(X_test)

accuracy=accuracy_score(y_test,y_pred)
print("accuracy:",accuracy)
print("正答率(%):",accuracy*100,"%")

accuracy: 0.9047619047619048
正答率(%): 90.47619047619048 %


次にベイズ最適化を行い、精度が上がるのか検証を行う。

# ランダムフォレスト（ベイズ最適化）

## ベイズ最適化

    **gp_params: ガウシアンプロセス（GP）に関するハイパーパラメータを指定します。alphaはガウシアンプロセスの正則化パラメータで、ここでは1e-5（0.00001）に設定されています。ガウシアンプロセスは、ベイズ最適化アルゴリズムの一部として使用され、目的関数の不確かさをモデル化するのに役立ちます。

In [37]:
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from bayes_opt import BayesianOptimization

def randomforest_cv(n_estimators, min_samples_split, max_features):
    val = cross_val_score(
        RandomForestClassifier(
            n_estimators=int(n_estimators),
            min_samples_split=int(min_samples_split),
            max_features=max_features,
            random_state=42
        ),
        X_train, y_train,
        scoring = 'accuracy',
        cv = 3, # 3-fold
        n_jobs = -1 # use all CPUs
    ).mean()
    return val

randomforest_cv_bo = BayesianOptimization(
    randomforest_cv,
    {'n_estimators': (10, 250),
    'min_samples_split': (2, 25),
    'max_features': (0.1, 0.999)}
)

gp_params = {"alpha": 1e-5}
randomforest_cv_bo.maximize(n_iter=50, **gp_params)

best_params=randomforest_cv_bo.max["params"]
print(best_params)


|   iter    |  target   | max_fe... | min_sa... | n_esti... |
-------------------------------------------------------------


C:\Users\morik\AppData\Local\Temp\ipykernel_13120\2790233001.py:28: DeprecationWarning: 
Passing acquisition function parameters or gaussian process parameters to maximize
is no longer supported, and will cause an error in future releases. Instead,
please use the "set_gp_params" method to set the gp params, and pass an instance
 of bayes_opt.util.UtilityFunction using the acquisition_function argument

  randomforest_cv_bo.maximize(n_iter=50, **gp_params)


| 1         | 0.9277    | 0.8308    | 3.253     | 239.9     |
| 2         | 0.9277    | 0.5572    | 11.97     | 29.93     |
| 3         | 0.9039    | 0.393     | 6.724     | 168.2     |
| 4         | 0.9396    | 0.8548    | 6.869     | 158.2     |
| 5         | 0.7844    | 0.1846    | 15.64     | 43.28     |
| 6         | 0.9396    | 0.7189    | 8.17      | 159.1     |
| 7         | 0.9277    | 0.8938    | 8.609     | 17.62     |
| 8         | 0.9277    | 0.975     | 15.43     | 147.3     |
| 9         | 0.9277    | 0.999     | 24.27     | 18.22     |
| 10        | 0.9277    | 0.999     | 25.0      | 160.2     |
| 11        | 0.8677    | 0.1738    | 20.78     | 244.7     |
| 12        | 0.9277    | 0.1       | 2.0       | 141.8     |
| 13        | 0.9039    | 0.8738    | 2.154     | 223.8     |
| 14        | 0.9039    | 0.1       | 12.7      | 128.9     |
| 15        | 0.8801    | 0.1229    | 2.044     | 25.45     |
| 16        | 0.8682    | 0.1       | 16.54     | 22.74     |
| 17    

## モデル作成と予測

In [38]:
best_model=RandomForestClassifier(
    n_estimators=int(best_params["n_estimators"]),
    min_samples_split=int(best_params["min_samples_split"]),
    max_features=best_params["max_features"]
)

best_model.fit(X_train,y_train)

y_pred=best_model.predict(X_test)

accuracy=accuracy_score(y_test,y_pred)
print("accuracy:",accuracy)
print("正答率(%):",accuracy*100,"%")

accuracy: 1.0
正答率(%): 100.0 %
